# 음식점 위치 정보

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json
import base64

In [ ]:
df_market = pd.read_csv('./files/음식테마거리.csv')
df_market = df_market.dropna(subset=['지번주소'])
df_market = df_market[['지번주소','식당명','영업신고증업태명']]
#df_market.to_csv('음식테마거리_null제거.csv')

In [ ]:
df = pd.read_csv('./files/음식테마거리_null제거.csv')
df

In [ ]:
# split 및 for문으로 해결
df = pd.read_csv('./files/음식테마거리.csv')
df = df.dropna()
list = []
df['지번주소'].str.split(" ")
for i in df.index:
    df_sigungu = df['지번주소'].str.split(" ")[i][1]
    list.append(df_sigungu)

In [ ]:
df_count = df.groupby('시군구').count()
df_count

In [ ]:
df_market = pd.read_excel('./files/부산_시군구_좌표.xlsx', usecols=[0,1,5,6])
df_merge = pd.merge(df_count, df_market, how = 'left', on = '시군구')
df_merge

In [ ]:
df_count = df.groupby('시군구').count()
df_market = pd.read_excel('./files/부산_시군구_좌표.xlsx', usecols=[0,1,5,6])
df_merge = pd.merge(df_count, df_market, how = 'left', on = '시군구')
df_merge = df_merge[['시군구', 'Unnamed: 0','위도','경도']]
df_merge.sort_values('Unnamed: 0', ascending=False)

df_merge = df_merge.reset_index()
df_merge = df_merge.drop(columns='index')
df_merge

In [ ]:
#df_merge = df_merge.reset_index()

In [ ]:
import numpy as np
import pandas as pd
import folium
import json

import warnings

warnings.filterwarnings(action='ignore')


#한글깨짐 해결
!pip3 install git+https://github.com/python-visualization/branca.git@master

# 음식테마거리 시각화

In [ ]:
#세븐브릿지 좌표
df_bridge = pd.read_csv('./files/좌표/세븐브릿지_좌표.txt')
df_bridge.columns=['브릿지','위도','경도']

#트래킹 좌표
df_mountain = pd.read_csv('./files/좌표/5대트래킹_좌표.txt')
df_mountain.columns=['트래킹','위도','경도']

#해수욕장 좌표
df_beach = pd.read_excel('./files/해수욕장좌표.xlsx')
df_beach


# 음식테마거리
df_count = df.groupby('시군구').count()
df_market = pd.read_excel('./files/부산_시군구_좌표.xlsx', usecols=[0,1,5,6])
df_market = df_market.drop(0)
df_merge = pd.merge(df_count, df_market, how = 'left', on = '시군구')
df_merge = df_merge[['시군구', 'Unnamed: 0','위도','경도']]
df_merge = df_merge.drop(6)
    
    
# 부산 위경도 확인
busan = folium.Map(
    location = [35.159144, 129.048293],
    tiles = "CartoDB positron",
    zoom_start = 11
)

# 부산 geojson style 지정
file_path = "./maps/부산_시군구.geojson"
busan_sgg_get = json.load(open(file_path, encoding="utf-8"))


print(len(busan_sgg_get['features']))
      
for i in range (len(busan_sgg_get['features'])):
    result = busan_sgg_get['features'][i]['properties']
    print(result)


def style_function(feature) :
    df = {
        "opacity" : 0.5,
        "weight" : 0.6,
        "color" : "yellow",
        "fillOpacity" : 0.5
    }
    return df

folium.GeoJson(
    busan_sgg_get,
    style_function = style_function
).add_to(busan)


tooltip = 'Click!'

#지도에 세븐브릿지 표시
for i in df_bridge.index:
    
    folium.Marker([df_bridge.loc[i, "위도"], df_bridge.loc[i, "경도"]],
                popup=f'<div style="width:200px"><strong>{df_bridge.iloc[i]["브릿지"]}</strong><br>\
                <img width="200px" src = ./image/{i}_bridge.jpg></div>',
                icon = folium.Icon(color='blue', icon='ok-circle'),
                tooltip = tooltip).add_to(busan)
    



#지도에 트래킹 표시
for j in range(len(df_mountain)):
    
    folium.Marker([df_mountain.loc[j, "위도"],df_mountain.loc[j, "경도"]],
                 popup=f'<div style="width:200px"><strong>{df_mountain.iloc[j]["트래킹"]}</strong><br>\
                 <img width="200px" src = ./image/{j}_mountain.jpg></div>',
                 icon = folium.Icon(color='green', icon='ok-circle'),
              
                 tooltip = tooltip
                 ).add_to(busan)

    
#지도에 해수욕장 표시
for i in df_bridge.index:
    
    folium.Marker([df_beach.loc[i, "위도"], df_beach.loc[i, "경도"]],
                popup=f'<div style="width:200px"><strong>{df_beach.iloc[i]["해수욕장"]}</strong><br>\
                <img width="200px" src = ./image/{i}_beach.jpg></div>',
                icon = folium.Icon(color='red', icon='ok-circle'),
                
                tooltip = tooltip).add_to(busan) 



#지도에 음식테마거리 표시
for item in range(len(df_merge)):
       
    folium.CircleMarker([df_merge.iloc[item]["위도"], df_merge.iloc[item]["경도"]],
                 color = 'green',
                 fill = True,
                 radius=df_merge.iloc[item]['Unnamed: 0']/150,
                 popup=f'<div style="width:80px"><strong>{df_merge.iloc[item]["시군구"]}</strong><br>\
                        개수 : {df_merge.iloc[item]["Unnamed: 0"]}</div>').add_to(busan)
                    

busan.save('index_food.html')

In [ ]:
df_merge.iloc[0]["시군구"]

In [ ]:
df_merge.index

# 부산 음식테마거리 음식점 종류별 개수

In [ ]:
df_food = df.iloc[:,[2,3,4,5]]
df_food_group = df_food.groupby('영업신고증업태명').count().iloc[:,[0]]
df_food_count = df_food_group.sort_values('식당명', ascending=False)
df_food_count = df_food_count.reset_index(drop=False)
df_food_count.head(1)

In [ ]:
#df_food_count = df_food_count.reset_index(drop=False)

In [ ]:
# Basic Bar Chart
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'

plt.figure(figsize=(15, 12))
sns.countplot(y=df_food['영업신고증업태명'], order = df_food['영업신고증업태명'].value_counts().index)
plt.yticks(fontsize=12)
plt.title('부산시 업종별 개수')
plt.show()

# 부산 음식점 시군구별 개수

In [ ]:
k = []
for x in df_merge.index:
    k.append(df_merge['시군구'][x])
k

In [ ]:
import numpy as np
# 그림 사이즈, 바 굵기 조정
fig, ax = plt.subplots(figsize=(16,8))
bar_width = 0.4 ## Figure 생성 
fig.set_facecolor('white') ## Figure 배경색 지정
plt.rc('font',family ='Malgun Gothic',size=15)
#데이터넣기
plt.title('부산 시군구별 음식점 개수')
#plt.ylabel('전통시장 개수')
index = np.arange(16)

# 각 연도별로 3개 샵의 bar를 순서대로 나타내는 과정, 각 그래프는 0.25의 간격을 두고 그려짐
plt.bar(index, df_merge['Unnamed: 0.1'], bar_width, alpha=0.4, color='red', label='음식점개수')

plt.legend(loc="upper right")

# x축 위치를 정 가운데로 조정하고 x축의 텍스트
# list = ['강서구','금정구','기장군','남구','동구','동래구','부산진구','북구','사상구','사하구','서구','수영구','연제구','영도구','중구','해운대구']
# plt.xticks(list)
plt.xticks([i for i in df_merge.index], k)

In [ ]:
# pie 그래프
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(16,8))
bar_width = 0.4 ## Figure 생성 
fig.set_facecolor('white') ## Figure 배경색 지정
plt.rc('font',family ='Malgun Gothic',size=15)
#데이터넣기
plt.title('부산 시군구별 음식점 개수')
# ratio = [34, 32, 16, 18]
# labels = ['Apple', 'Banana', 'Melon', 'Grapes']
# plt.bar(index, df_merge['Unnamed: 0.1'], bar_width, alpha=0.4, color='red', label='음식점개수')
explode = [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
colors = ['silver', 'gold', 'whitesmoke', 'lightgray','lightgray','pink','pink','lightgray','lightgray','lightgray','lightgray','silver','lightgray','lightgray','lightgray','lightgray']

plt.pie(df_merge['Unnamed: 0.1'], labels=k, autopct='%.1f%%', explode=explode, shadow=True, colors=colors)
plt.show()

In [ ]:
df_merge